![title](Chapter-2.png "Header")
___
# Chapter 2 -  Machine Learning Based Recommendation Systems
## Segment 2 - Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [26]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [27]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [28]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [29]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [30]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [31]:
filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

### Building a Utility Matrix

In [32]:
rating_crosstab = combined_movies_data.pivot_table(values='rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


### Transposing the Matrix

In [33]:
rating_crosstab.shape

(943, 1664)

In [34]:
X = rating_crosstab.T
X.shape

(1664, 943)

### Decomposing the Matrix

In [35]:
SVD = TruncatedSVD(n_components=12, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(1664, 12)

In [40]:
from pandas import Series, DataFrame
DataFrame(resultant_matrix)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.039994,0.659884,0.045689,0.814447,0.167474,-0.983164,-0.319829,0.306784,-0.030842,-0.753561,-0.301530,-0.536511
1,0.436584,-0.257261,0.352955,-0.668667,-0.293370,-0.006088,-0.231560,-0.568051,0.336801,-0.247432,0.352607,-0.083584
2,12.543744,5.669234,-4.907812,5.022113,7.881279,-0.813447,-0.121968,-4.842374,2.150923,3.802348,0.602189,-0.831987
3,25.663725,-12.267626,6.077408,3.115259,0.713566,1.886036,-3.659231,-0.388550,1.370892,-5.671582,1.250848,-1.607138
4,3.636418,4.217210,2.633261,1.655672,-3.200588,-1.942051,-0.653495,-0.830418,2.308042,-0.201487,-0.742995,-1.589686
5,13.239159,8.403065,1.542896,-7.235866,1.066606,-3.661391,-5.005779,-2.140963,-1.827323,-2.301714,-2.398330,-2.794711
6,15.009596,-6.028979,-4.609465,5.190691,-1.384635,1.347183,-1.769085,-6.536553,0.004986,3.732616,0.686199,-2.443007
7,47.942963,-17.765585,5.037396,-1.677213,-3.119208,9.414925,1.169901,0.451649,3.797822,5.022636,-5.265951,1.042847
8,0.191216,0.010013,0.238734,0.100751,-0.128566,-0.150668,-0.049059,-0.161192,-0.146883,-0.092101,0.095335,-0.073190
9,11.607555,-9.467347,5.015958,2.713852,-0.100173,3.481127,-2.386874,-8.175943,-0.005870,-2.029841,0.690934,-0.746915


### Generating a Correlation Matrix

In [41]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(1664, 1664)

### Isolating Star Wars From the Correlation Matrix

In [42]:
movie_names = rating_crosstab.columns
movies_list = list(movie_names)

star_wars = movies_list.index('Star Wars (1977)')
star_wars

1398

In [16]:
corr_star_wars = corr_mat[1398]
corr_star_wars.shape

(1664,)

### Recommending a Highly Correlated Movie

In [17]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.9)])


['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Star Wars (1977)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [18]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.95)])

['Return of the Jedi (1983)', 'Star Wars (1977)']